In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from sqlalchemy import create_engine

In [2]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install selenium webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [15]:
url = "https://www.pcmag.com/picks/best-iphone-apps?test_uuid=01nEKitrPAQmw6jRcSPVArQ&test_variant=A"

In [16]:
response = requests.get(url)

In [17]:
response

<Response [200]>

In [18]:
response.text

'<!doctype html>\n<html lang="en">\n\n<head>\n    \n        <meta charset="utf-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1">\n    <meta name="csrf-token" content="">\n    <meta name="application-name" content="PCMag">\n    <title>The Best iPhone Apps for 2025 | PCMag</title>\n<meta name="description" content="Your iPhone 16 needs apps, but how do you find the best ones in the packed Apple App Store? We help you decide with 117 top picks in 12 key categories.">\n\n    <meta name="zd_ptax_version" content="PCMagV3" />\n    <meta name="zd_ptax" content="apple,software-services,mobile-apps,iphone-apps" />\n\n    \n    <link href="https://www.pcmag.com/picks/best-iphone-apps" rel="canonical" />\n\n\n\n<meta name="robots" content="max-image-preview:large">\n\n<meta name="article:published_time" content="2025-04-14T14:44:19+00:00">    <meta property="fb:app_id" content="141810985954037" />\n<meta property="og:site_name" content="PCMAG" />\n\n    <meta property=

In [19]:
soup = BeautifulSoup(response.text)

In [20]:
soup

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="" name="csrf-token"/>
<meta content="PCMag" name="application-name"/>
<title>The Best iPhone Apps for 2025 | PCMag</title>
<meta content="Your iPhone 16 needs apps, but how do you find the best ones in the packed Apple App Store? We help you decide with 117 top picks in 12 key categories." name="description"/>
<meta content="PCMagV3" name="zd_ptax_version"/>
<meta content="apple,software-services,mobile-apps,iphone-apps" name="zd_ptax"/>
<link href="https://www.pcmag.com/picks/best-iphone-apps" rel="canonical"/>
<meta content="max-image-preview:large" name="robots"/>
<meta content="2025-04-14T14:44:19+00:00" name="article:published_time"/> <meta content="141810985954037" property="fb:app_id"/>
<meta content="PCMAG" property="og:site_name"/>
<meta content="article" property="og:type"/>
<meta content="https://www.pcmag.com/picks/best-iphone-

In [27]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd

# Headless browser setup
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Launch browser
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get("https://www.pcmag.com/picks/best-iphone-apps")

# Wait for JS to finish loading
driver.implicitly_wait(5)

# Grab full rendered HTML
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
driver.quit()

# Grab the app cards
cards = soup.find_all("div", class_="grouped-roundup-product-card")

# Extract info
data = []
for card in cards:
    name_tag = card.find("h2")
    desc_tag = card.find("p")
    review_link = card.find("a", href=True)

    data.append({
        "App Name": name_tag.get_text(strip=True) if name_tag else None,
        "Description": desc_tag.get_text(strip=True) if desc_tag else None,
        "Review Link": f"https://www.pcmag.com{review_link['href']}" if review_link else None
    })

df = pd.DataFrame(data)
df.head()


,App Name,Description,Review Link
0,Facebook Messenger,Whatever your take on the social network of re...,https://www.pcmag.comhttps://cc.pcmag.com/v1/o...
1,Gmail,Google's email app is a wonderful communicatio...,https://www.pcmag.comhttps://cc.pcmag.com/v1/o...
2,Kik,Many messaging apps require you to give up you...,https://www.pcmag.comhttps://cc.pcmag.com/v1/o...
3,Microsoft Outlook,Microsoft Outlook's light and flexible mobile ...,https://www.pcmag.comhttps://cc.pcmag.com/v1/o...
4,Skype,Skype is one of the best free communication to...,https://www.pcmag.comhttps://cc.pcmag.com/v1/o...


In [25]:
user = 'postgres'
password = 'isba_4715'
host = 'isba-dev-02.cxcoakgsc7ha.us-east-1.rds.amazonaws.com'
port = 5432
database = 'postgres'

engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')

In [26]:
df.to_sql(name= 'Best iPhone Apps 2025', con=engine, if_exists='replace', index=False)

107